In [2]:
import pandas as pd
from datetime import datetime
import numpy as np


In [3]:
# colunas que vao para o arquivo FULL final
columns = [
    "timestamp",
    "STEERING_WHEEL",
    "THROTTLE",
    "BRAKE",
    "ECU_MODE_ID",
    "TORQUE_GAIN",
    "TORQUE_REF_LEFT_MOTOR",
    "TORQUE_REF_RIGHT_MOTOR",
    "LEFT_MOTOR_SPEED",
    "RIGHT_MOTOR_SPEED",
    "ACCEL_LONGITUDINAL",
    "STACK_1_CELL_1"
    "STACK_1_CELL_2",
    "STACK_1_CELL_3",
    "STACK_1_CELL_4",
    "STACK_2_CELL_1",
    "STACK_2_CELL_2",
    "STACK_2_CELL_3",
    "STACK_2_CELL_4",
    "STACK_3_CELL_1",
    "STACK_3_CELL_2",
    "STACK_3_CELL_3",
    "STACK_3_CELL_4",
    "STACK_4_CELL_1",
    "STACK_4_CELL_2",
    "STACK_4_CELL_3",
    "STACK_4_CELL_4",
    "STACK_5_CELL_1",
    "STACK_5_CELL_2",
    "STACK_5_CELL_3",
    "STACK_5_CELL_4",
    "STACK_6_CELL_1",
    "STACK_6_CELL_2",
    "STACK_6_CELL_3",
    "STACK_6_CELL_4",
    "MAX_VOLTAGE",
    "MIN_VOLTAGE",
    "TOTAL_VOLTAGE",
    "SHUNT_CURRENT",
    "BMS_MODE_ID",
    "BMS_ERROR_ID",
    "AIR_P",
    "AIR_N",
    "STACK_ERROR_UNDER",
    "CELL_ERROR_UNDER",
    "MIN_CELL_VOLTAGE"
]

In [4]:
# timestamps de interesse
# nome: (inicio, fim)
interest_timestamps = {
    # miguelito
    "aceleracao1_miguelito": ("2025-12-08 15:38:00", "2025-12-08 15:38:30"), # aceleração bem leve, estranho
    "autocross1_miguelito":  ("2025-12-08 15:28:25", "2025-12-08 15:29:00"), # incompleto, shunt ruim
    "autocross2_miguelito":  ("2025-12-08 15:39:20", "2025-12-08 15:40:00"), # OK
    "autocross3_miguelito":  ("2025-12-08 15:43:30", "2025-12-08 15:45:30"), # OK
    "aceleracao2_miguelito": ("2025-12-08 15:47:40", "2025-12-08 15:48:00"), # com falha de subtensão
    "autocross4_miguelito": ("2025-12-08 15:49:00", "2025-12-08 15:50:00"), # OK
    "aceleracao3_miguelito": ("2025-12-08 16:13:15", "2025-12-08 16:13:35"), # bom
    "autocross5_miguelito": ("2025-12-08 16:16:00", "2025-12-08 16:17:30"), # OK
    "aceleracao4_miguelito": ("2025-12-08 16:21:40", "2025-12-08 16:22:15"), # com falha de subtensão
    "aceleracao5_miguelito": ("2025-12-08 16:39:00", "2025-12-08 16:39:30"), # com falha de subtensão

    # mike
    "aceleracao1_mike": ("2025-12-08 17:01:00", "2025-12-08 17:01:20"), # com falha de subtensão
    "autocross1_mike": ("2025-12-08 17:03:00", "2025-12-08 17:04:15"), 
    "aceleracao2_mike": ("2025-12-08 17:07:20", "2025-12-08 17:07:45"), # com falha de subtensão
    "aceleracao3_mike": ("2025-12-08 17:10:40", "2025-12-08 17:11:05"), # 50 km/h, publico
    "autocross2_mike": ("2025-12-08 17:21:00", "2025-12-08 17:24:30"), 
    "aceleracao4_mike": ("2025-12-08 17:26:45", "2025-12-08 17:27:00"), # com falha de subtensão
    "autocross3_mike": ("2025-12-08 17:28:15", "2025-12-08 17:31:15"), 
    "aceleracao5_mike": ("2025-12-08 17:33:45", "2025-12-08 17:34:00"), # com falha de subtensão
    "aceleracao6_mike": ("2025-12-08 17:40:50", "2025-12-08 17:41:10"), # com falha de subtensão

    # pedro
    "autocross1_pedro": ("2025-12-08 18:01:45", "2025-12-08 18:04:30"), # tensoes estranhas nas stacks
    "aceleracao1_pedro": ("2025-12-08 18:16:25", "2025-12-08 18:16:45"), # com falha de subtensão
}

In [5]:
full_df = pd.DataFrame([], columns=columns)
full_df

,timestamp,STEERING_WHEEL,THROTTLE,BRAKE,ECU_MODE_ID,TORQUE_GAIN,TORQUE_REF_LEFT_MOTOR,TORQUE_REF_RIGHT_MOTOR,LEFT_MOTOR_SPEED,RIGHT_MOTOR_SPEED,...,MIN_VOLTAGE,TOTAL_VOLTAGE,SHUNT_CURRENT,BMS_MODE_ID,BMS_ERROR_ID,AIR_P,AIR_N,STACK_ERROR_UNDER,CELL_ERROR_UNDER,MIN_CELL_VOLTAGE


In [6]:
file_names = [
    "76_DRIVER_INPUTS.csv",
    "77_ECU_MODE.csv",
    "78_TORQUE_REFERENCE.csv",
    "79_WHEEL_SPEED.csv",
    "80_MOTOR_SPEED.csv",
    "85_LEFT_MOTOR_TRACTIVE.csv",
    "95_RIGHT_MOTOR_TRACTIVE.csv",
    "259_ACCELEROMETER.csv",
    "260_GYROSCOPE.csv",
    "261_ELETROBUILD_TEMPERATURE.csv",
    "300_STACK_1_VOLTAGE.csv",
    "301_STACK_2_VOLTAGE.csv",
    "302_STACK_3_VOLTAGE.csv",
    "303_STACK_4_VOLTAGE.csv",
    "304_STACK_5_VOLTAGE.csv",
    "305_STACK_6_VOLTAGE.csv",
    "306_ACCUMULADOR_PARAMS.csv",
    "307_BMS_PARAMS.csv",
    "308_BMS_ERROR_FLAGS.csv",
]

folder_names = [
    "./data/dados_telemetria/teste_odonto_08122025/teste1_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste2_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste4_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste5_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste7_longer",
    "./data/dados_telemetria/teste_odonto_08122025/teste8",
]


In [7]:
full_folder = "./data/dados_telemetria/teste_odonto_08122025/full"

for file_name in file_names:
    print(file_name + "...")
    dfs_list = []
    for folder in folder_names:
        dfs_list.append(pd.read_csv(f"{folder}/{file_name}"))

    combined_df = pd.concat(dfs_list, ignore_index=True)
    combined_df.to_csv(f"{full_folder}/{"FULL_" + file_name}", index=False)

76_DRIVER_INPUTS.csv...
77_ECU_MODE.csv...
78_TORQUE_REFERENCE.csv...
79_WHEEL_SPEED.csv...
80_MOTOR_SPEED.csv...
85_LEFT_MOTOR_TRACTIVE.csv...
95_RIGHT_MOTOR_TRACTIVE.csv...
259_ACCELEROMETER.csv...
260_GYROSCOPE.csv...
261_ELETROBUILD_TEMPERATURE.csv...
300_STACK_1_VOLTAGE.csv...
301_STACK_2_VOLTAGE.csv...
302_STACK_3_VOLTAGE.csv...
303_STACK_4_VOLTAGE.csv...
304_STACK_5_VOLTAGE.csv...
305_STACK_6_VOLTAGE.csv...
306_ACCUMULADOR_PARAMS.csv...
307_BMS_PARAMS.csv...
308_BMS_ERROR_FLAGS.csv...


In [8]:
# agora junta as que nao estao no inicial
extra_file_names = [
    "311_STACK_OVER_ERROR.csv",
    "312_STACK_ERROR_UNDER.csv",
]

extra_folder_names = [
    "./data/dados_telemetria/teste_odonto_08122025/teste5_0812",
    "./data/dados_telemetria/teste_odonto_08122025/teste7_longer",
]

full_folder = "./data/dados_telemetria/teste_odonto_08122025/full"

for file_name in extra_file_names:
    print(file_name + "...")
    dfs_list = []
    for folder in extra_folder_names:
        dfs_list.append(pd.read_csv(f"{folder}/{file_name}"))

    combined_df = pd.concat(dfs_list, ignore_index=True)
    combined_df.to_csv(f"{full_folder}/{"FULL_" + file_name}", index=False)

311_STACK_OVER_ERROR.csv...
312_STACK_ERROR_UNDER.csv...


In [21]:
# quais colunas precisa refatorar para colocar no arquivo final FULL.csv
# (old_name, new_name)
columns_rename_map = {
    "FULL_77_ECU_MODE.csv": [("MODE_ID", "ECU_MODE_ID")],
    "FULL_85_LEFT_MOTOR_TRACTIVE.csv": [("SPEED", "LEFT_MOTOR_SPEED")],
    "FULL_95_RIGHT_MOTOR_TRACTIVE.csv": [("SPEED", "RIGHT_MOTOR_SPEED")],
    "FULL_259_ACCELEROMETER.csv": [("ACCEL_Z", "ACCEL_LONGITUDINAL")],
    "FULL_300_STACK_1_VOLTAGE.csv": [
        ("CEL_1", "STACK_1_CELL_1"), ("CEL_2", "STACK_1_CELL_2"), ("CEL_3", "STACK_1_CELL_3"), ("CEL_4", "STACK_1_CELL_4")
    ],
    "FULL_301_STACK_2_VOLTAGE.csv": [
        ("CEL_1", "STACK_2_CELL_1"), ("CEL_2", "STACK_2_CELL_2"), ("CEL_3", "STACK_2_CELL_3"), ("CEL_4", "STACK_2_CELL_4")
    ],
    "FULL_302_STACK_3_VOLTAGE.csv": [
        ("CEL_1", "STACK_3_CELL_1"), ("CEL_2", "STACK_3_CELL_2"), ("CEL_3", "STACK_3_CELL_3"), ("CEL_4", "STACK_3_CELL_4")
    ],
    "FULL_303_STACK_4_VOLTAGE.csv": [
        ("CEL_1", "STACK_4_CELL_1"), ("CEL_2", "STACK_4_CELL_2"), ("CEL_3", "STACK_4_CELL_3"), ("CEL_4", "STACK_4_CELL_4")
    ],
    "FULL_304_STACK_5_VOLTAGE.csv": [
        ("CEL_1", "STACK_5_CELL_1"), ("CEL_2", "STACK_5_CELL_2"), ("CEL_3", "STACK_5_CELL_3"), ("CEL_4", "STACK_5_CELL_4")
    ],
    "FULL_305_STACK_6_VOLTAGE.csv": [
        ("CEL_1", "STACK_6_CELL_1"), ("CEL_2", "STACK_6_CELL_2"), ("CEL_3", "STACK_6_CELL_3"), ("CEL_4", "STACK_6_CELL_4")
    ],
    "FULL_306_ACCUMULADOR_PARAMS.csv": [("SHUNT_VOLTAGE", "SHUNT_CURRENT")],
    "FULL_307_BMS_PARAMS.csv": [("MODE_ID", "BMS_MODE_ID"), ("ERROR_ID", "BMS_ERROR_ID")],
}

# quais colunas vou efetivamente usar dos full individuais
file_columns_map = {
    'FULL_76_DRIVER_INPUTS.csv': ['STEERING_WHEEL', 'THROTTLE', 'BRAKE'],
    'FULL_77_ECU_MODE.csv': ['ECU_MODE_ID', 'TORQUE_GAIN_ID']
}

In [26]:
# passa pelos fulls: remove timestamps que estão fora dos ranges
def is_in_range(timestamp):
    for key in interest_timestamps:
        start_date, end_date = interest_timestamps[key]

        start_timestamp = datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000
        end_timestamp = datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S").timestamp() * 1000

        if timestamp >= start_timestamp and timestamp <= end_timestamp:
            return True, key
        
    return False, None

full_folder = "./data/dados_telemetria/teste_odonto_08122025/full"
full_file_names = file_names + extra_file_names

for i in range(len(full_file_names)):
    full_file_names[i] = "FULL_" + full_file_names[i]

for file_name in full_file_names:
    print(file_name + "...")

    # verifica se existe na lista do file_columns_map
    if file_name not in file_columns_map:
        continue

    # le o arquivo atual
    df = pd.read_csv(f'{full_folder}/{file_name}')

    # monta o dicionario de dados desse arquivo
    data_row = {}

    for column in columns:
        data_row[column] = []

    # varre os timestamps
    for i in range (df.shape[0]):
        curr_timestamp = df.iloc[i, 1] 
        is_range, key = is_in_range(timestamp=curr_timestamp)
        
        if is_range:
            # pega as colunas desse arquivo
            columns_to_add = file_columns_map[file_name]

            # para cada coluna do full_df
            for col in full_df.columns:
                if col == 'timestamp': 
                    data_row['timestamp'].append(curr_timestamp)
                    continue 

                # verifica se essa coluna tem dados para adicionar
                if col in columns_to_add:
                    # se o arquivo atual esta na lista de refatoração
                    if file_name in columns_rename_map:
                        refactor_list = columns_rename_map[file_name]
                        result = next((item for item in refactor_list if item[1] == col), None)

                        if result == None:
                            raise ValueError("Column Not Found")
                        
                        old_name, new_name = result
                        data_row[new_name].append(df.iloc[i][old_name])
                    else:
                        data_row[col].append(df.iloc[i][col])
                else:
                    # sem dados para adicionar
                    data_row[col].append(np.nan)

    # salva o dicionario na full_df
    new_rows = pd.DataFrame(data_row)
    full_df = pd.concat([full_df, new_rows], ignore_index=True)

# TODO: ORDENAR O DATAFRAME ANTES DE SALVAR NO CSV
sorted_full_df = full_df.sort_values(by='timestamp')

# salva o full_df para um CSV novo
sorted_full_df.to_csv(f'{full_folder}/full_df.csv')

FULL_76_DRIVER_INPUTS.csv...
FULL_77_ECU_MODE.csv...
FULL_78_TORQUE_REFERENCE.csv...
FULL_79_WHEEL_SPEED.csv...
FULL_80_MOTOR_SPEED.csv...
FULL_85_LEFT_MOTOR_TRACTIVE.csv...
FULL_95_RIGHT_MOTOR_TRACTIVE.csv...
FULL_259_ACCELEROMETER.csv...
FULL_260_GYROSCOPE.csv...
FULL_261_ELETROBUILD_TEMPERATURE.csv...
FULL_300_STACK_1_VOLTAGE.csv...
FULL_301_STACK_2_VOLTAGE.csv...
FULL_302_STACK_3_VOLTAGE.csv...
FULL_303_STACK_4_VOLTAGE.csv...
FULL_304_STACK_5_VOLTAGE.csv...
FULL_305_STACK_6_VOLTAGE.csv...
FULL_306_ACCUMULADOR_PARAMS.csv...
FULL_307_BMS_PARAMS.csv...
FULL_308_BMS_ERROR_FLAGS.csv...
FULL_311_STACK_OVER_ERROR.csv...
FULL_312_STACK_ERROR_UNDER.csv...
